<a href="https://colab.research.google.com/github/kerlonn/Projeto-CDIA/blob/main/Projeto_ICD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Caminho para o arquivo CSV no Google Drive
file_path = '/content/sample_data/GCB2022v27_MtCO2_flat.csv'  # Se não funcionar veja se o caminho esta certo.

In [ ]:
# Função para carregar, preparar e limpar o DataFrame
def load_and_clean_data(file_name):
    # Carrega o arquivo CSV no DataFrame
    df = pd.read_csv(file_name)

    # Remove apenas as linhas onde a coluna 'Total' possui valores NaN ou 0
    df_cleaned = df[df['Total'].notna() & (df['Total'] > 0)]

    # Retorna o DataFrame limpo
    return df_cleaned


In [ ]:
# Atribui o Dataframe filtrado para uma variável
df_cleaned = load_and_clean_data(file_path)
# Exibe as 10 primeiras linhas
df_cleaned.head(10)

In [ ]:
# Filtrando os dados para o Brasil
df_brazil = df_cleaned[df_cleaned['Country'] == 'Brazil']
df_brazil

In [ ]:
# Calcular os maiores emissores, removendo o "Global"
top_5_countries = df_cleaned[df_cleaned['Country'] != 'Global'].groupby("Country")['Total'].max().sort_values(ascending=False).head(5)
top_5_countries

In [ ]:
# Função para calcular medidas de centralidade e dispersão
def calculate_statistics(df):
    stats = df.describe()  # Calcula estatísticas básicas
    iqr = df.quantile(0.75) - df.quantile(0.25)  # Intervalo Interquartil (IQR)
    stats.loc['IQR'] = iqr # Adiciona o valor do IQR como uma nova linha (IQR) ao DataFrame stats gerado por describe().
    return stats

In [ ]:
# Calcular medidas de centralidade e dispersão para cada dataset
cleaned_stats = calculate_statistics(df_cleaned[['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']])
brazil_stats = calculate_statistics(df_brazil[['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']])
top_emitters_stats = calculate_statistics(df_top_emitters[['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']])


In [ ]:
# Vizualização das medidas de centralidade e dispersão das variáveis disponíveis
print("Medidas de Centralidade e Dispersão - Dados Globais (df_cleaned)")
print(cleaned_stats)
print("\n\nEstatísticas Descritivas dos Cinco Maiores Emissores (df_top_emitters):")
print(top_emitters_stats.describe())
print("\nMedidas de Centralidade e Dispersão - Brasil (df_brazil):")
print(brazil_stats.head())



In [ ]:
# Função para criar gráficos de barras
def create_bar_plot(df):
    plt.figure(figsize=(15, 6))  # Aumentar a largura do gráfico
    sns.barplot(x='Country', y='Total', data=df)
    plt.title('Emissões Totais de Carbono por País')
    plt.xticks(rotation=90, ha='right')
    plt.ylabel('Emissões Totais')
    plt.xlabel('País')
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.8)  # Aumentar o espaço na parte inferior
    plt.show()

In [ ]:
## Criação do boxplot do Brazil
# Filtragem dos dados do Brasil para remover colunas não numéricas ou com valores inválidos
df_brazil_numeric = df_brazil[['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']].apply(pd.to_numeric, errors='coerce')

# Remover colunas que possuem apenas valores NaN após a conversão
df_brazil_numeric_cleaned = df_brazil_numeric.dropna(axis=1, how='all')

# Gerar o boxplot para o dataset filtrado de dados do Brasil
create_boxplots(df_brazil_numeric_cleaned, 'Brasil - Emissões de CO₂ (Variáveis Numéricas Filtradas)')


In [ ]:
## Função teste

# Função para filtrar e criar boxplots, removendo a coluna 'Year' e ajustando o eixo Y
def create_filtered_boxplot(df, title, y_limit=None):
    # Remover a coluna 'Year' se estiver presente
    if 'Year' in df.columns:
        df = df.drop(columns=['Year'])

    # Filtrar apenas colunas numéricas e converter valores inválidos para NaN
    df_numeric = df.apply(pd.to_numeric, errors='coerce')

    # Remover colunas que possuem apenas valores NaN
    df_numeric_cleaned = df_numeric.dropna(axis=1, how='all')

    # Gerar o boxplot
    plt.figure(figsize=(14, 8))
    df_numeric_cleaned.boxplot()
    plt.title(title)
    plt.xticks(rotation=45)
    plt.ylabel('Emissões de CO₂')

    # Definir o limite do eixo Y, se fornecido
    if y_limit:
        plt.ylim(y_limit)

    plt.grid(True)
    plt.show()

# Testar a função ajustada nos três datasets
# Definir um limite de eixo Y mais apropriado para o Brasil
create_filtered_boxplot(df_brazil, 'Brasil - Emissões de CO₂ (Variáveis Numéricas Filtradas)', y_limit=(0, 500))

# Aplicar o ajuste nos outros dois datasets com limite automático
create_filtered_boxplot(df_cleaned, 'Dados Globais - Emissões de CO₂ (Variáveis Numéricas Filtradas)')
create_filtered_boxplot(df_top_emitters, 'Cinco Maiores Emissores - Emissões de CO₂ (Variáveis Numéricas Filtradas)')
